In [ ]:
import pymongo
import os
import json
from bson import ObjectId
import asyncio
import aiohttp

async def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """
    async with aiohttp.ClientSession() as session:
        # Generate embedding for the user query
        url = f"http://20.193.130.111:5000/preprocess"
        headers = {"Content-Type": "application/json"}
        async with session.post(url, headers=headers, data=json.dumps(data)) as response:
            print("s3")
            json_data = await response.json()
            query_embedding = get_embedding(user_query)

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 4,  # Return top 4 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "fullplot": 1,  # Include the plot field
                "title": 1,  # Include the title field
                "genres": 1,  # Include the genres field
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)
